<a href="https://colab.research.google.com/github/raverinpuff/arina-ML/blob/main/%D1%80%D0%B5%D0%BA%D1%83%D1%80%D1%80%D0%B5%D0%BD%D1%82%D0%BD%D0%B0%D1%8F_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# устанавливаем библиотеки и обновляем datasets, чтобы всё работало

!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

!pip install datasets -q
!pip install -U datasets huggingface_hub fsspec
from datasets import load_dataset

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [2]:
dataset = load_dataset("SocialGrep/one-million-reddit-confessions") # будем работать с датасетом признавашек из реддита

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

one-million-reddit-confessions.csv:   0%|          | 0.00/779M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [9]:
data = dataset['train']['title'][:1000] # возьмём только заголовок (текстов значительно меньше) и 1000 вхождений, чтобы хватило вычислительных мощностей

In [10]:
# Инициализируем токенизатор
tokenizer = Tokenizer()

# Обучаем токенизатор на заголовках
tokenizer.fit_on_texts(data)

# Преобразуем заголовки в последовательности чисел
sequences = tokenizer.texts_to_sequences(data)

# Создаем входные и выходные данные
X = []
y = []
for seq in sequences:
  for i in range(1, len(seq)):
    X.append(seq[:i])
    y.append(seq[i])

In [11]:
X[:10], y[:10]

([[1],
  [1, 14],
  [1, 14, 399],
  [1, 14, 399, 5],
  [1, 14, 399, 5, 581],
  [1, 14, 399, 5, 581, 33],
  [1, 14, 399, 5, 581, 33, 2],
  [1, 14, 399, 5, 581, 33, 2, 108],
  [1, 14, 399, 5, 581, 33, 2, 108, 1222],
  [1]],
 [14, 399, 5, 581, 33, 2, 108, 1222, 357, 582])

In [12]:
# Преобразуем списки в массивы numpy
X = np.asarray(X, dtype="object")
y = np.array(y)

# Дополняем последовательности до одинаковой длины
X = pad_sequences(X)

# Преобразуем y в one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

In [13]:
# Создаем модель
model = Sequential()

# Добавляем слой Embedding
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X.shape[1]))

# Добавляем слой LSTM
model.add(LSTM(150, return_sequences=False))

# Добавляем полносвязный слой
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

# Компилируем модель
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Выводим информацию о модели
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Обучаем модель
history = model.fit(X, y, epochs=50, batch_size=64, validation_split=0.2)

Epoch 1/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 36s 175ms/step - accuracy: 0.0374 - loss: 6.9886 - val_accuracy: 0.0469 - val_loss: 6.4767
Epoch 2/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 33s 172ms/step - accuracy: 0.0481 - loss: 6.2395 - val_accuracy: 0.0489 - val_loss: 6.5309
Epoch 3/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 33s 172ms/step - accuracy: 0.0557 - loss: 6.1207 - val_accuracy: 0.0616 - val_loss: 6.5565
Epoch 4/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 33s 172ms/step - accuracy: 0.0809 - loss: 5.9248 - val_accuracy: 0.0759 - val_loss: 6.4447
Epoch 5/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 33s 171ms/step - accuracy: 0.1034 - loss: 5.6731 - val_accuracy: 0.0870 - val_loss: 6.3634
Epoch 6/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 33s 172ms/step - accuracy: 0.1214 - loss: 5.4255 - val_accuracy: 0.1043 - val_loss: 6.3735
Epoch 7/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 33s 172ms/step - accuracy: 0.1286 - loss: 5.2685 - val_accuracy: 0.1020 - val_loss: 6.3264
Epoch 8/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 33s 172ms/step - accuracy: 0.1391 - loss: 5

In [15]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 67, 100)        │       286,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2866)           │       432,766 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,609,900 (9.96 MB)

 Trainable params: 869,966 (3.32 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,739,934 (6.64 MB)

In [19]:
# Функция для генерации текста
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Генерируем новый заголовок
generated_text = generate_text("My", 15, X.shape[1])
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
My husband and i don’t know what to do about it now i am done there


In [ ]:
# ура, получилось!

# Сохраняем модель
model.save('reddit_confession_generator.keras')